<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/03_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install cryptos
! pip install eciespy


Cloning into 'Colegio_Invisible'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 282 (delta 69), reused 23 (delta 23), pack-reused 186
Receiving objects: 100% (282/282), 12.16 MiB | 34.01 MiB/s, done.
Resolving deltas: 100% (181/181), done.
     |████████████████████████████████| 69 kB 3.4 MB/s 
     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 555 kB 49.7 MB/s 
     |████████████████████████████████| 104 kB 57.0 MB/s 
     |████████████████████████████████| 481 kB 49.6 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1231117 sha256=682953734d868a61447b20c3d6822763af52bcbf00f7bfd3c6fc093f1aa02e04
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
Successfully built cytoolz


In [ ]:
 #! rm -r Colegio_Invisible

# Intermediate Cryptography and Python Scripts

## Outline

In this deeper dive we cover:

- asymmetric cryptography
- python scripts

## Goals

By the end of the exposition you will be able to :

- understand the features of **asymmetric cryptography**
- generate a new **private key** object
- access the byte or hex value of the private key
- import a previously generated **private key** from bytes
- generate and save a password protected private key file using a python script
- derive a **public key** from the private key
- access the byte or hex value of the public key
- import a previously generated **public key** from bytes
- derive and save a password protected public key file using a python script
- dump the contents of an AES protected key file
- use the keyboard to type and save AES password protected key files using a python script 
- understand ECDSA digital signature
- **sign** a message with a private key
- **verify** a message signature with a public key
- understand ECIES
- **ECIES encrypt** with a public key
- **ECIES decrypt** with a private key


# Asymmetric Cryptography

Asymmetric cryptography, or public-key cryptography, is a system that uses pairs of keys: public keys (which may be known to others), and private keys (which may never be known by any except the owner).

- Public keys are generated from private keys
- Public keys are shown to others
- Private keys are held private

<br></br>
<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/asymmetric.png" width="500"/>
</div>

The two keys have the opposite action. If one locks the other unlocks.

- one key is used to encrypt (lock) and the other is used to decrypt (unlock)
- when the public key is used to encrypt (lock) only the private key can decrypt (unlock). this means only one entity can decrypt (unlock) but anyone can encrypt (lock)
- when the private key is used to encrypt (lock) only the public key can decrypt (unlock). this means anyone can decrypt (unlock) but only one entity can encrypt (lock)

**ENCRYPTION**
For encryption and decryption:
- the recipients public key is used to encrypt
- the recipients private key is used to decrypt

**DIGITAL SIGNATURE**
For signature and verification
- the sender's private key is used the sign the hash of a text to generate a signature
- the sender's public key is used to turn the signature back into the hash and compared to the hash of the text. if the hashes match then the text/signature/pubkey fit and the signature is deemed valid.

**ECC** and RSA are different kinds of asymmetric cryptography.
Elliptic-curve cryptography (ECC) is an approach to public-key cryptography based on the algebraic structure of elliptic curves over finite fields. ECC allows smaller keys compared to RSA.

**secp256k1** refers to the parameters of the elliptic curve used in Ethereum's, Bitcoin's, and their derivatives' public-key cryptography.

## Python Implementation

We will be using the asymmetric cryptography library developed by Weiliang Li.
- https://github.com/kigawas
- https://github.com/ecies/py

## Imports

- import the required libraries
- have a look at the scripts
- make a directory for keys

In [2]:
import ecies
import eth_keys

In [3]:
! ls Colegio_Invisible/scripts

aes_decrypt.py	ecc_encrypt.py	 ecc_keydump.py		ecc_verify.py
aes_encrypt.py	ecc_generate.py  ecc_pubkey_extract.py	sha_256.py
ecc_decrypt.py	ecc_keyboard.py  ecc_sign.py


In [9]:
! mkdir keys

# Key Management

This journey begins by managing keys and keyfiles.

In this implementation of ECC:
- private keys are 32 bytes (64 hex numerals)
- public keys are 64 bytes (128 hex numerals)





## Private Keys

Private keys must be kept secret. Never ever share them.

Security goals:
- private keys must be unique
- private keys must be secret

### Generate Private Key

There are various methods for creating keys. We will start by using the `generate_eth_key()` method. This produces a unique key using a psuedo-random number generator.

The method produces a private key object.

In [ ]:
#this generates a private key object
privKey1 = ecies.utils.generate_eth_key()
privKey1, type(privKey1)

In [ ]:
#this generates a private key
privKey2 = ecies.utils.generate_eth_key()
privKey2, type(privKey2)

### Dump Private Key Bytes

Here we get the actual 32 bytes of data from the private key object.

In [ ]:
#here we can dump the contents of the private key as bytes
privKey1_bytes=privKey1.to_bytes()
privKey1_bytes,privKey1_bytes.hex()

### Import Previously Generated Private Key From Bytes

Here we create a private key object from 32 bytes of data.

In [ ]:
#here we are generating a key object from previously generated bytes
privKey1_copy=eth_keys.keys.PrivateKey(privKey1_bytes)
privKey1_copy,type(privKey1_copy)

### Generate Private Key and Save in AES Password Protected File Using a Python Script

The script `ecc_generate.py` is a command line tools for producing a private key. The 32 bytes are stored in an AES password protected file.

In [ ]:
! ls Colegio_Invisible/scripts
! cat Colegio_Invisible/scripts/ecc_generate.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_generate.py keys/my_privkey.enc

## Public Keys

### Derive Public Key From Private Key

In [ ]:
pubKey1=privKey1.public_key
pubKey1

In [ ]:
pubKey2=privKey2.public_key
pubKey2

### Dump Public Key Bytes

In [ ]:
pubKey1_bytes=pubKey1.to_bytes()
pubKey1_bytes

### Import Previously Generated Private Key From Bytes

In [ ]:
pubKey1_import=eth_keys.keys.PublicKey(pubKey1_bytes)
pubKey1_import,type(pubKey1_import)

### Extracting Public Key and Saving to AES Password Protected File Using Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_pubkey_extract.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py keys/my_privkey.enc keys/my_pubkey.enc

## Input / Output

### Dumping the Contents of an AES Password Protected Key File Using a Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_keydump.py 

In [ ]:
! python Colegio_Invisible/scripts/ecc_keydump.py keys/my_pubkey.enc

In [ ]:
pubkey=!{'python Colegio_Invisible/scripts/ecc_keydump.py keys/my_pubkey.enc'}
pubkey

In [ ]:
bytes.fromhex(pubkey[-1])

### Keyboard Input Contents of an AES Password Protected Key File Using a Python Script

In [4]:
! cat Colegio_Invisible/scripts/ecc_keyboard.py 

##################################################################################
# this script will generate an ECC private or public key from keyboard input
# the key is saved to an AEC encrypted (password protected) file
#
# run at the terminal using the following
# > python ecc_keyboard.py <KEY_PATH> <KEY_PASSWORD> <KEY_HEX>
# <KEY_PATH>: path to keyboard input key file
# <KEY_PASSWORD>: password for key file
# <KEY_HEX>: hex key (64 hex values for private key and 128 hex values for public key)
#
#EXAMPLE:
#>python ecc_keyboard.py keys\privkey.bin password123 0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef
#
# IF <KEY_PASSWORD> AND <KEY_HEX> ARE NOT INCLUDED USER WILL BE PROMPTED TO ENTER THEM SECURELY
##################################################################################

from sys import argv
import getpass
import ecies
import eth_keys
import hashlib
from Crypto.PublicKey import ECC
#from Crypto.Random import get_random_bytes


if len(argv)==4:
    _,

In [10]:
! python Colegio_Invisible/scripts/ecc_keyboard.py keys/tecla_privkey.bin password123 0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef

Key generation complete
Key encrypted and written to binary file: keys/tecla_privkey.bin
3117957cbcf6cf6503f450bd57e38572ecdae662fd3408c70924ad34fa5bead69e7a0f2673b1d0c2645f2786ff7de07dee815454ca68fafe028562e37f37e4ef


In [11]:
! python Colegio_Invisible/scripts/ecc_keyboard.py keys/tecla_pubkey.bin password123 0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef

Key generation complete
Key encrypted and written to binary file: keys/tecla_pubkey.bin
a28b3586a7a238ef5f6f203172dbd74d0713a22f1ecb2dd95be711be268ed57ab44926efdcf9cc803a4c8637672f69c724d68fd0fee0672312dd48d941062e9cee60275c5df8de442c5420ed0986e16107cb7950632905552e850723ae15ff2e


---
# Digital Signature ECDSA


Digital signature is a means by which we ensure the authenticity and integrity of a message.

- authenticity relates to the source of a message
- integrity relates to the fact that a message has not been tampered with by adding, removing or changing its content

**ECDSA** (Elliptic Curve Digital Signature Algoritm) is an algorithm for performing digital signature using elliptic curve cryptography. 

- signature is performed using a private key and a text in byte form
- the signature algortitm is used generate a signature 
- verification is performed using the triplet signature/text/publickey
 
https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm

*SECURITY NOTE:* the text and signature can be used to recover the public key value

<br></br>
<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/signature.png" width="500"/>
</div>


## Sign Message

In [ ]:
### lets make two messages
message_A=b'this is message A'
message_B=b'this is message B'

In [ ]:
### now lets criss-cross the two keys and two messages
signature_1A=privKey1.sign_msg(message_A)
signature_1B=privKey1.sign_msg(message_B)
signature_2A=privKey2.sign_msg(message_A)
signature_2B=privKey2.sign_msg(message_B)

## Dump the Contents of the Signature as Bytes

In [ ]:
sig_bytes=signature_1A.to_bytes()
sig_bytes

In [ ]:
new_sig=eth_keys.datatypes.Signature(sig_bytes)
new_sig, type(new_sig)

## Sign File with Private Key using Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_sign.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_sign.py  keys/my_privkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg ic_img.sig

In [ ]:
! python Colegio_Invisible/scripts/sha_256.py Colegio_Invisible/img/colegio_invisible.jpeg

## Verify Signature

In [ ]:
#if signature, message and key match then True
signature_1A.verify_msg(message_A,pubKey1)

In [ ]:
#if message does not match
signature_1A.verify_msg(message_B,pubKey1)

In [ ]:
#if public key does not match
signature_1A.verify_msg(message_A,pubKey2)

In [ ]:
signature_2B.verify_msg(message_B,pubKey2)

## Verify Signature/Message/PublicKey Triplet with Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_verify.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_verify.py keys/my_pubkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg ic_img.sig

# Hybrid Encryption ECIES

ECIES stands for Elliptic Curve Integrated Encryption Scheme. It is a hybrid scheme where we use both ECC and AES.

The process goes as follows:

***SENDING...***

1. The sender generates a one-time-use AES encryption key (session key)
2. The sender encrypts the plain text with the **AES session key** and produces a cipher text
3. The sender then encrypts the AES session key with the **reciever public key**
4. The sender then packages the encrypted file and the encrypted key together

***RECIEVING...***

5. The **reciever private key** is used to decrypt the **encrypted AES session key**
6. The **decrypted AES session key** is used to decrypt the ciper text...recovering the plain text

---
---

## Encrypt

In [ ]:
ciphertext_1A=ecies.encrypt(pubKey1.to_hex(),msg=message_A)
ciphertext_1A

In [ ]:
ciphertext_2B=ecies.encrypt(pubKey2.to_hex(),msg=message_B)
ciphertext_2B

## Encrypt File with Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_encrypt.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_encrypt.py keys/my_pubkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg colegio_invisible.enc

## Decrypt

In [ ]:
ecies.decrypt(privKey1.to_hex(),ciphertext_1A)

In [ ]:
ecies.decrypt(privKey2.to_hex(),ciphertext_2B)

In [ ]:
#if we use the wrong key...
ecies.decrypt(privKey1.to_hex(),ciphertext_2B)

## Decrypt Using Python Script

In [ ]:
! python Colegio_Invisible/scripts/ecc_decrypt.py keys/my_privkey.enc '' colegio_invisible.enc colegio_invisible_dec.jpeg

In [ ]:
from IPython.display import Image
display(Image('colegio_invisible_dec.jpeg'))